# Constraint statistics


In [7]:
def per_row_ztest(row, statistic="z"):
    if statistic == "z":
        i = 0
    elif statistic == "p":
        i = 1

    stat = proportions_ztest(
        count=row["n_obs"],
        nobs=row["n_pos"],
        value=row["prop_exp"],
        alternative="smaller",
        prop_var=row["prop_exp"],
    )[i]
    
    return stat

## Get FDR-adjusted P-values
Calculate separately for whole-transcripts and constrained regions

In [10]:
def fdr_adjustment(df, regions=["transcript"], csq="nonsense"):
    """Get FDR-adjusted P-values for a given region and variant consequence."""
    # Mask regions and consequences
    m1 = df.region.isin(regions)
    m2 = df.csq == csq

    # Filter the dataframe and drop cases without a P-value
    p = df.loc[m1 & m2, ["region", "p"]].dropna().copy()

    # FDR adjustment
    p["fdr_p"] = fdr(pvals=p["p"])[1]

    return p


r1 = ["transcript"]
r2 = ["distal_nmd", "nmd_target", "long_exon"]

fdr_p = pd.concat([fdr_adjustment(df, regions=r) for r in [r1, r2]])
df = df.join(fdr_p["fdr_p"])